## IRIS Dataset
picture and text published from http://suruchifialoke.com/2016-10-13-machine-learning-tutorial-iris-classification/
![title](iris.png)

Three Iris varieties were used in the Iris flower data set outlined by Ronald Fisher in his famous 1936 paper “The use of multiple measurements in taxonomic problems as an example of linear discriminant analysis” PDF. Since then his paper has been cited over 2000 times and the data set has been used by almost every data science beginner.

The data set consists of:

150 samples
3 labels: species of Iris (Iris setosa, Iris virginica and Iris versicolor)

4 features: length and the width of the sepals and petals, in centimetres

In [ ]:
import numpy as np
import pandas as pd
from sklearn import datasets
import matplotlib.pyplot as plt
import seaborn as sns
random_state = 42

### Load Dataset


sklearn libraray comes with IRIS data pre-loaded, which we would convert into a pandas dataframe


In [ ]:
def get_iris_dataset():
    iris = datasets.load_iris()
    iris_df = pd.DataFrame(iris.data, columns= iris.feature_names)
    iris_df['target'] = iris.target
    print('shape', iris_df.shape)
    iris_df.head( )
    return iris_df, iris.target_names



 Read data an print first five rows

In [ ]:
iris_df, target_names = get_iris_dataset()
iris_df.head()

Print data info

In [ ]:
iris_df.info()

The target field in data represents the  species of flower which are mapped as mapped as
<br>setosa : 0, 
<br>versicolor: 1 
<br>virginica: 2. 



In [ ]:
# print Target Names
print(target_names)


We can see  that there are 50  samples of each species

In [ ]:
iris_df.target.value_counts()

### Prepare Data
We will only sepal length as feature and check if we can predict the flower species based on sepal length only .
We will only predict two classes of flowers 

setosa : 0
<br>versicolor: 1 

In [ ]:
# select the rows with 0 and 1 target values
data = iris_df[iris_df.target.isin([0,1]) ]
feature = 'sepal length (cm)'

#y is the target value and X is feature column
y   = data['target']
X   = data[feature]



### Split Data into  training and test set
Split the data into training set and test set such that number of samples in training and test set are in 80:20 ratio. The model will be trained on training set and then evalauted on test set. We use stratified sampling so that the ratio of each class of flower is same in training and test set

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = random_state, stratify = y )

X_train = X_train.values.reshape(-1,1)
X_test = X_test.values.reshape(-1,1)


print('Train Shape', X_train.shape, 'Test Shape', X_test.shape )
y_train.value_counts()


### Plot Training Data
Plot the sepal length and corresponding class of flower

In [ ]:
def display_classes(X_train, y_train):
    train_df = pd.DataFrame()
    train_df[feature] = X_train.ravel()   
    train_df['target'] = y_train.values
    plt.figure(figsize = (10, 6))
#     ax = sns.scatterplot(x= feature, y= 'target', hue="target", 
#                        style ='target', palette="Set2", data= train_df)
    train_0 = train_df[train_df['target'] == 0]
    train_1 = train_df[train_df['target'] == 1]
    plt.xlabel(feature)
    plt.ylabel('class')
    plt.scatter(x =train_0[feature], y=  train_0['target'], label = 'setosa', color = 'r' )
    plt.scatter(x =train_1[feature], y=  train_1['target'] , label ='versicolor', color = 'g')
    plt.legend( )
    plt.show()


In [ ]:
display_classes(X_train, y_train)

### Train the Model 
Train the Logistic Regression model using traning set data.

The model documentaion can be accessed using link 
http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(random_state= random_state, C = 1, solver = 'lbfgs')
model.fit(X_train, y_train)

### Predict on Training set
The prediction are probabilities for positive class

In [ ]:
y_prob = model.predict_proba(X_train)[:,-1]
y_prob[:5]


### Plot the featue vs probabilties for Training Data
The blue line is probability threshold for classifying target as 0 or 1
The data above blue line are predicted as class 1 and below the line are predicted as class 0. We can see that there are some points below blue lines(in green) which are predicted as 1 but actually belong to class 0 (False Negatives)

In [ ]:
def display_prob(X, y, y_prob):
    train_df = pd.DataFrame()
    train_df[feature] = X.ravel()
    train_df['pred'] = y_prob   
    train_df['Actual Classes'] = y.values
    plt.figure(figsize=(10,6))
#     ax = sns.scatterplot(x= train_df[feature], y =  y_prob, hue="Actual Classes", 
#                           palette="Set2", data= train_df  )
    train_0 = train_df[train_df['Actual Classes'] == 0]
    train_1 = train_df[train_df['Actual Classes'] == 1]
    plt.xlabel(feature)
    plt.ylabel('Probabilty')
    plt.scatter(x =train_0[feature], y=  train_0['pred'], label = '0_setosa', color = 'r' )
    plt.scatter(x =train_1[feature], y=  train_1['pred'] , label ='1_versicolor', color = 'g')
    plt.axhline(y = 0.5, color = 'b')
    plt.legend()
    
    plt.show()
    


In [ ]:
display_prob(X_train, y_train, y_prob )

### Plot the featue vs probabilties for Test data
Similarly for test data there are points in both classes 0 and 1 which are incorrectly predicted. Can you find those points?


In [ ]:
y_prob = model.predict_proba(X_test)[:,-1]
display_prob(X_test, y_test, y_prob )

### Predict Classes
The predict_proba in previous section method ouputs probabilities. Lets predict classes instead of probalities and check.
accuracy scores.
Even though our training accuracy is 0.92 the test accuarcy is low.  We can set petal length as feature which would have given accuracy of 1 on both training and test set

From the confusion  matrix for Test set
<br>Number of correct Predictions = TN + TP = 6 + 8 = 14
<br>False Positives = 4
<br>False Negatives = 2 
<br>Accuracy = TN + TP /(TP + TN + FP + FN) = 14/20 = 0.7

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix
y_pred = model.predict(X_train)
print('Training Accuracy', accuracy_score(y_train, y_pred))

y_pred = model.predict(X_test)
print('\nTest Accuracy', accuracy_score(y_test, y_pred))
print('Test set confusion matrix\n', confusion_matrix(y_test, y_pred))